# Классификация текстов: спам-фильтр для SMS

Возьмём открытый датасет с SMS-сообщениями, размеченными на спам ("spam") и не спам ("ham"), и построим на нем классификатор текстов на эти два класса, оценим его качество с помощью кросс-валидации, протестируем его работу на отдельных примерах, и посмотрим, что будет происходить с качеством, если менять параметры модели.

In [69]:
%pylab inline
import pandas as pd
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
import warnings
from itertools import product

Populating the interactive namespace from numpy and matplotlib


C:\Users\Admin\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['clf', 'product']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


Считаем датасет.

In [70]:
reviews = pd.read_csv('SMSSpamCollection.txt', '\t', header=None)

Подготовим для дальнейшей работы два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используем 1 для "спама" и 0 для "не спама".

In [71]:
y = reviews[0]

In [72]:
y = y.replace(['ham','spam'], [0,1])

In [73]:
x = reviews[1]

In [74]:
x

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                 Will ü b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: 1, Length: 5572, dtype: object

Используя sklearn.feature_extraction.text.CountVectorizer со стандартными настройками, получим из списка текстов матрицу признаков X.

In [75]:
from sklearn.feature_extraction.text import CountVectorizer

In [76]:
vectorizer = CountVectorizer()

In [77]:
X = vectorizer.fit_transform(x)

In [78]:
X

<5572x8713 sparse matrix of type '<class 'numpy.int64'>'
	with 74169 stored elements in Compressed Sparse Row format>

Оценим качество классификации текстов с помощью LogisticRegression() с параметрами по умолчанию, используя sklearn.cross_validation.cross_val_score и посчитав среднее арифметическое качества на отдельных fold'ах. Установим random_state=2. Параметр cv зададим равным 10. В качестве метрики качества будем использовать f1-меру.

In [79]:
from sklearn import cross_validation, datasets, linear_model, metrics

In [80]:
logistic_regressor = linear_model.LogisticRegression(random_state = 2)
logistic_regressor.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=2, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [81]:
scoring = cross_validation.cross_val_score(logistic_regressor, X, y, scoring = 'f1', 
                                                  cv = 10)
print (scoring.mean())

0.9326402983610631


Теперь обучим классификатор на всей выборке и спрогнозируем с его помощью класс для следующих сообщений:

In [82]:
demo = vectorizer.transform(["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
                                  "FreeMsg: Txt: claim your reward of 3 hours talk time","Have you visited the last lecture on physics?",
                                  "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
                                  "Only 99$"])

In [83]:
print(logistic_regressor.predict(demo))

[1 1 0 0 0]


Поэкспериментируем: попробуем добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Зададим в CountVectorizer параметр ngram_range=(2,2), затем ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях измерим получившееся в кросс-валидации значение f1-меры.

In [84]:
vectorizer22 = CountVectorizer(ngram_range=(2,2))
X22 = vectorizer22.fit_transform(x)
logistic_regressor.fit(X22, y)
scoring = cross_validation.cross_val_score(logistic_regressor, X22, y, scoring = 'f1', 
                                                  cv = 10)
print (scoring.mean())

0.8224220664187133


In [85]:
vectorizer33 = CountVectorizer(ngram_range=(3,3))
X33 = vectorizer33.fit_transform(x)
logistic_regressor.fit(X33, y)
scoring = cross_validation.cross_val_score(logistic_regressor, X33, y, scoring = 'f1', 
                                                  cv = 10)
print (scoring.mean())

0.7250161555467377


In [86]:
vectorizer13 = CountVectorizer(ngram_range=(1,3))
X13 = vectorizer13.fit_transform(x)
logistic_regressor.fit(X13, y)
scoring = cross_validation.cross_val_score(logistic_regressor, X13, y, scoring = 'f1', 
                                                  cv = 10)
print (scoring.mean())

0.9251382558648837


Статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

Повторим аналогичный предыдущему пункту эксперимент, используя вместо логистической регрессии MultinomialNB().

In [87]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()

In [88]:
scoring = cross_validation.cross_val_score(clf, X, y, scoring = 'f1', 
                                                  cv = 10)
print (scoring.mean())

0.9277303556851543


In [89]:
clf.fit(X22, y)
scoring = cross_validation.cross_val_score(clf, X22, y, scoring = 'f1', 
                                                  cv = 10)
print (scoring.mean())

0.6455015177985443


In [90]:
clf.fit(X33, y)
scoring = cross_validation.cross_val_score(clf, X33, y, scoring = 'f1', 
                                                  cv = 10)
print (scoring.mean())

0.37871948524573595


In [91]:
clf.fit(X13, y)
scoring = cross_validation.cross_val_score(clf, X13, y, scoring = 'f1', 
                                                  cv = 10)
print (scoring.mean())

0.8884859656061002


Обратим внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

Попробуем использовать в логистической регрессии в качестве признаков Tf*idf из TfidfVectorizer на униграммах

In [92]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [93]:
tfvectorizer = TfidfVectorizer()

In [94]:
Xtf = tfvectorizer.fit_transform(x)

In [95]:
logistic_regressor.fit(Xtf, y)
scoring = cross_validation.cross_val_score(logistic_regressor, Xtf, y, scoring = 'f1', 
                                                  cv = 10)
print (scoring.mean())

0.8528599554172456


Качество на кросс-валидации понизилось по сравнению с CountVectorizer на униграммах.